# Python

This is an ipython notebook 

In [1]:
import timeit
import simplejson
from statistics import mean

In [2]:
fibonanchi__cache={}
def fibonanchi_cached(n):
    if n not in fibonanchi__cache:
        if n <= 1: fibonanchi__cache[n] = n
        else:      fibonanchi__cache[n] = fibonanchi_cached(n-1) + fibonanchi_cached(n-2) 
    return fibonanchi__cache[n]

def fibonanchi_uncached(n):
    if n <= 1: return n
    else:      return fibonanchi_uncached(n-1) + fibonanchi_uncached(n-2)
    
def fibonanchi_looped(n):
    fibonanchi__loop_cache = [0,1]
    if n >= len(fibonanchi__loop_cache): 
        for i in range(len(fibonanchi__loop_cache), n+1):
            fibonanchi__loop_cache.append( fibonanchi__loop_cache[i-1] + fibonanchi__loop_cache[i-2] )
    return fibonanchi__loop_cache[n]
    
def fibonanchi_range_cached(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_cached(n) for n in range(start,end) ]
    
def fibonanchi_range_uncached(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_uncached(n) for n in range(start,end) ]

def fibonanchi_range_looped(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_looped(n) for n in range(start,end) ]

In [3]:
fibonanchi_looped(300)

222232244629420445529739893461909967206666939096499764990979600

In [4]:
fibonanchi_range_cached(30) == fibonanchi_range_uncached(30) == fibonanchi_range_looped(30)

True

In [5]:
fibonanchi_cached(30) == fibonanchi_uncached(30) == fibonanchi_looped(30)

True

#### Performance Tests

How high we can fibonanchi in a reasonable time?

In [6]:
funcs   = { "fibonanchi_uncached": fibonanchi_uncached, "fibonanchi_cached": fibonanchi_cached, "fibonanchi_looped": fibonanchi_looped }
timings = {}

# Note: fibonanchi_uncached(64) takes an inordinate amount of time (got bored waiting)
for name, func in funcs.items():
    try:
        time = 0
        n    = 1
        timings[name] = {}
        while time <= 0.5:
            time = timeit.timeit(lambda: func(2**n), number=1)
            timings[name][2**n] = time
            n    = n+1
            # print(name, 2**n, time)
    except Exception as exception: 
        # print(exception)
        pass
timings

{'fibonanchi_cached': {2: 4.043999069835991e-06,
  4: 2.642000254127197e-06,
  8: 2.642000254127197e-06,
  16: 2.4929995561251417e-06,
  32: 7.720000212430023e-06,
  64: 3.427099909458775e-05,
  128: 7.386799916275777e-05,
  256: 0.0001367679997201776,
  512: 0.000350010001056944,
  1024: 0.0006335969992505852,
  2048: 0.0014873780000925763,
  4096: 0.0035401929999352433},
 'fibonanchi_looped': {2: 1.0523999662837014e-05,
  4: 6.770998879801482e-06,
  8: 7.761998858768493e-06,
  16: 1.0530000508879311e-05,
  32: 1.630600127100479e-05,
  64: 2.8077000024495646e-05,
  128: 5.1483000788721256e-05,
  256: 0.00016133399913087487,
  512: 0.00021935200129519217,
  1024: 0.00042332800148869865,
  2048: 0.0009211030010192189,
  4096: 0.0021267600004648557,
  8192: 0.008512929000062286,
  16384: 0.030237303999456344,
  32768: 0.10378496900011669,
  65536: 0.40394193300016923,
  131072: 1.4152641870005027},
 'fibonanchi_uncached': {2: 8.697999874129891e-06,
  4: 8.122000508592464e-06,
  8: 2.7495

Now inspect relative timings for each doubling of the fibonanchi number

In [7]:
timings_diff = {}
for name, func in funcs.items():
    timings_diff[name] = {}
    for n in range(0,100):
        if 2**n in timings[name] and 2**(n+1) in timings[name]:
            timings_diff[name][2**(n+1)] = timings[name][2**(n+1)] / timings[name][2**n] 

timings_diff

{'fibonanchi_cached': {4: 0.6533137640494924,
  8: 1.0,
  16: 0.9436030720400976,
  32: 3.0966713144663314,
  64: 4.439248465227733,
  128: 2.155408395269789,
  256: 1.8515189428486958,
  512: 2.559151276417377,
  1024: 1.8102254145232373,
  2048: 2.347514274612787,
  4096: 2.3801568933484947},
 'fibonanchi_looped': {4: 0.6433864592101465,
  8: 1.1463594953357998,
  16: 1.3566093863804027,
  32: 1.548528061062763,
  64: 1.7218813832930309,
  128: 1.833636098721557,
  256: 3.1337334005250805,
  512: 1.359614231822598,
  1024: 1.9299026176606728,
  2048: 2.175861265449054,
  4096: 2.3089274468887333,
  8192: 4.002768999887893,
  16384: 3.5519271920669264,
  32768: 3.4323486314118052,
  65536: 3.8921043855562076,
  131072: 3.5036327535718104},
 'fibonanchi_uncached': {4: 0.933777951957599,
  8: 3.3853726565283733,
  16: 52.58059656319742,
  32: 1248.2270616772819}}

In [8]:
{ name: mean(timings_diff[name].values()) for name in timings_diff.keys() }

{'fibonanchi_cached': 2.1124374375276394,
 'fibonanchi_looped': 2.34632636305278,
 'fibonanchi_uncached': 326.2817022122413}

fibonanchi_uncached() 
- grows exponentially due having to recompute the entire recursion tree. 
- It quickly becomes comutationally infeasable to calculate higher numbers.

fibonanchi_cached()
- should be O(N), meaning 2x time per doubling of N
- mean() is 2.1 (close to 2x) and the doubling time seems to go both up and down
- this variance may be related to hash map bucket resizing operations 

fibonanchi_looped()
- should be O(N), meaning 2x time per doubling of N
- mean() is 2.5 and the doubling time seems to increase with higher N
- this may be an artifiact of arrays needing a full memcopy on each resize event to preserve a contigious memory space/
- thus to grow an array from 0 to N, may require log(N) resizing events that each need to copy an average of 1/2 N memory
- this makes fibonanchi_looped() = O(N) + μO(N log(N))